In [2]:
# Test if CUDA is available or Mac Performance Shaders (MPS) is available.
import torch

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available.")
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print("MPS backend is available.")
else:
    device = torch.device('cpu')
    print("No GPU available.")
print( f"Device: {device}" )

CUDA is available.
Device: cuda


In [1]:
import torch
from torchsummary import summary
from torchvision import models
from torch import nn
import numpy as np

rn18 = models.resnet18() # Load the model

children_counter = 0
for n, c in rn18.named_children():
    print(f"Child{children_counter:3d} -> Layer name: {n}")
    children_counter += 1
print( rn18._modules ) # Print the model structure

Child  0 -> Layer name: conv1
Child  1 -> Layer name: bn1
Child  2 -> Layer name: relu
Child  3 -> Layer name: maxpool
Child  4 -> Layer name: layer1
Child  5 -> Layer name: layer2
Child  6 -> Layer name: layer3
Child  7 -> Layer name: layer4
Child  8 -> Layer name: avgpool
Child  9 -> Layer name: fc
OrderedDict([('conv1', Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)), ('bn1', BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)), ('relu', ReLU(inplace=True)), ('maxpool', MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)), ('layer1', Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, mom

In [4]:
from torchinfo import summary
import torch

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available.")
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print("MPS backend is available.")
else:
    device = torch.device('cpu')
    print("No GPU available.")
print( f"Device: {device}" )

rn18 = models.resnet18().to(device) # Load the model to the device

summary(rn18, input_size=(1, 3, 224, 224), device=device) # Print the model summary

CUDA is available.
Device: cuda


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│

In [5]:
from torch import nn
from torchsummary import summary
from torchvision import models
from torchvision.models import resnet18, ResNet18_Weights

class new_model( nn.Module ):
    def __init__(self, output_layer):
        super().__init__()
        self.output_layer = output_layer
        self.pretrained_model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        self.children_list = []
        for n, c in self.pretrained_model.named_children():
            self.children_list.append(c)
            if n == output_layer:
                print(f"Output layer found: {n}")
                break
        self.net = nn.Sequential( *self.children_list )
        self.pretrained_model = None
    
    def forward(self, x):
        return self.net(x)

device = "cpu"
model = new_model(output_layer='layer1')
model = model.to(device)
summary(model, input_size=(3, 224, 224), device=device) # (3, 224, 224) means 3 channels, 224x224 pixels
            

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\fishd/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:03<00:00, 14.5MB/s]


Output layer found: layer1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
            